In [ ]:
# vietlott_env\Scripts\activate
# git add vietlott_645.ipynb
# git commit -m "Cập nhật mô hình 6/45"
# git push origin master

# Cài đặt thư viện 

In [14]:
! pip install pandas deap requests bayesian-optimization beautifulsoup4 seaborn selenium matplotlib sklearn tensorflow numpy scikit-learn xgboost lightgbm tensorflow-addons optuna imbalanced-learn torch schedule apscheduler pyarrow joblib


You should consider upgrading via the 'e:\python3_8_64bit\python.exe -m pip install --upgrade pip' command.


# Xử lý data

In [15]:
import pandas as pd
import os

# Tham số hóa đường dẫn
DATA_DIR = "data_645"
FILE_NAME = "E:/vietlot_mega\data_645/vietlott_645_filtered.csv"
FILE_PATH = os.path.join(DATA_DIR, FILE_NAME)

# Kiểm tra file tồn tại
if not os.path.exists(FILE_PATH):
    raise FileNotFoundError(f"Không tìm thấy file: {FILE_PATH}")

# Đọc dữ liệu
try:
    df = pd.read_csv(FILE_PATH, sep="\t")
except Exception as e:
    raise Exception(f"Lỗi khi đọc file {FILE_PATH}: {e}")

# Kiểm tra cột
expected_columns = ['Ngày Mở Thưởng', 'Kết Quả', 'Giải Jackpot']
if not all(col in df.columns for col in expected_columns):
    raise ValueError(f"Dữ liệu thiếu cột: {set(expected_columns) - set(df.columns)}")

# Kiểm tra dữ liệu thiếu
if df.isna().any().any():
    print("Cảnh báo: Dữ liệu có giá trị thiếu:")
    print(df.isna().sum())

# In thông tin cơ bản
print("Thông tin dữ liệu:")
print(f"- Số cột: {len(df.columns)}")
print(f"- Tên cột: {list(df.columns)}")
print(f"- Số dòng: {len(df)}")
print(f"- Dữ liệu mẫu (5 dòng đầu):\n{df.head()}")

Thông tin dữ liệu:
- Số cột: 3
- Tên cột: ['Ngày Mở Thưởng', 'Kết Quả', 'Giải Jackpot']
- Số dòng: 1356
- Dữ liệu mẫu (5 dòng đầu):
   Ngày Mở Thưởng            Kết Quả    Giải Jackpot
0  T4, 28/05/2025  17 22 23 28 31 41  29.736.250.000
1  CN, 25/05/2025  05 14 23 24 28 44  25.290.182.500
2  T6, 23/05/2025  02 04 15 16 29 43  23.051.056.000
3  T4, 21/05/2025  02 13 14 21 24 30  20.908.114.000
4  CN, 18/05/2025  06 07 17 27 30 42  18.835.213.500


In [16]:
import pandas as pd
import os

# Tham số hóa đường dẫn
DATA_DIR = "data_645"
FILE_NAME = "vietlott_645_filtered.csv"
FILE_PATH = os.path.join(DATA_DIR, FILE_NAME)

# Hàm kiểm tra tính hợp lệ của kết quả
def validate_result(result):
    try:
        if len(result) != 6:
            return False, "Phải có đúng 6 số"
        if not all(isinstance(x, int) and 1 <= x <= 45 for x in result):
            return False, "Số phải trong khoảng 1-45"
        return True, ""
    except:
        return False, "Định dạng không hợp lệ"

# Đọc dữ liệu
try:
    df = pd.read_csv(FILE_PATH, sep="\t", encoding="utf-8")
except Exception as e:
    raise Exception(f"Lỗi khi đọc file {FILE_PATH}: {e}")

# Kiểm tra và xử lý cột
if 'Kết Quả' in df.columns and 'Ngày Mở Thưởng' in df.columns and 'Giải Jackpot' in df.columns:
    # Chuyển đổi Kết Quả từ chuỗi phân cách bởi khoảng trắng sang danh sách số nguyên
    df['Kết Quả'] = df['Kết Quả'].apply(lambda x: [int(i) for i in x.split()] if isinstance(x, str) else x)
    
    # Kiểm tra tính hợp lệ của Kết Quả
    df['Valid_Result'] = df['Kết Quả'].apply(validate_result)
    invalid_results = df[~df['Valid_Result'].apply(lambda x: x[0])]
    if not invalid_results.empty:
        print("Cảnh báo: Phát hiện kết quả không hợp lệ:")
        print(invalid_results[['Ngày Mở Thưởng', 'Kết Quả', 'Valid_Result']])
    
    # Xử lý cột Ngày: Loại bỏ tiền tố ngày trong tuần (CN, T6, T4) và chuyển đổi định dạng
    try:
        # Tách phần ngày (bỏ tiền tố như CN, T6, T4)
        df['Ngày Mở Thưởng'] = df['Ngày Mở Thưởng'].str.split(',').str[-1].str.strip()
        # Chuyển đổi sang định dạng datetime
        df['Ngày Mở Thưởng'] = pd.to_datetime(df['Ngày Mở Thưởng'], format='%d/%m/%Y', dayfirst=True)
    except Exception as e:
        raise ValueError(f"Lỗi khi chuyển đổi cột Ngày: {e}")
    
    # Kiểm tra Jackpot
    df['Giải Jackpot'] = pd.to_numeric(df['Giải Jackpot'], errors='coerce')
    if df['Giải Jackpot'].isna().any():
        print("Cảnh báo: Có giá trị Jackpot không hợp lệ:")
        print(df[df['Giải Jackpot'].isna()][['Ngày Mở Thưởng', 'Giải Jackpot']])
    
    # Đổi tên cột cho thống nhất
    df = df.rename(columns={'Ngày Mở Thưởng': 'Ngày', 'Giải Jackpot': 'Jackpot'})
    
    # In dữ liệu mẫu
    print("Dữ liệu sau khi xử lý (5 dòng đầu):")
    print(df[['Ngày', 'Kết Quả', 'Jackpot']].head())
else:
    raise ValueError("Dữ liệu thiếu cột cần thiết")

Cảnh báo: Có giá trị Jackpot không hợp lệ:
     Ngày Mở Thưởng  Giải Jackpot
0        2025-05-28           NaN
1        2025-05-25           NaN
2        2025-05-23           NaN
3        2025-05-21           NaN
4        2025-05-18           NaN
...             ...           ...
1351     2016-08-05           NaN
1352     2016-08-03           NaN
1353     2016-07-31           NaN
1354     2016-07-29           NaN
1355     2016-07-27           NaN

[1356 rows x 2 columns]
Dữ liệu sau khi xử lý (5 dòng đầu):
        Ngày                   Kết Quả  Jackpot
0 2025-05-28  [17, 22, 23, 28, 31, 41]      NaN
1 2025-05-25   [5, 14, 23, 24, 28, 44]      NaN
2 2025-05-23    [2, 4, 15, 16, 29, 43]      NaN
3 2025-05-21   [2, 13, 14, 21, 24, 30]      NaN
4 2025-05-18    [6, 7, 17, 27, 30, 42]      NaN


In [17]:
df.to_csv(r"E:/vietlot_mega/data_645/vietlott_645_clean.csv", index=False, encoding="utf-8-sig")
print("✅ Đã ghi file thành công: vietlott_645_clean.csv")

✅ Đã ghi file thành công: vietlott_645_clean.csv


In [18]:
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from itertools import combinations
import numpy as np
import os

# Tham số hóa đường dẫn lưu biểu đồ
OUTPUT_DIR = "plots"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Kiểm tra tính hợp lệ của Kết Quả
if 'Valid_Result' not in df.columns:
    df['Valid_Result'] = df['Kết Quả'].apply(validate_result)
invalid_results = df[~df['Valid_Result'].apply(lambda x: x[0])]
if not invalid_results.empty:
    raise ValueError("Dữ liệu chứa kết quả không hợp lệ, vui lòng kiểm tra.")

# 1. Phân tích tần suất số
plt.figure(figsize=(15, 6))
all_numbers = np.concatenate(df['Kết Quả'].values)
freq = pd.Series(all_numbers).value_counts().sort_index()
sns.barplot(x=freq.index, y=freq.values, palette='viridis')
plt.title('Tần suất xuất hiện của các số (1-45)', fontsize=14)
plt.xlabel('Số', fontsize=12)
plt.ylabel('Tần suất', fontsize=12)
plt.xticks(np.arange(1, 46, 1), rotation=45, fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'number_frequency.png'), dpi=300)
plt.close()

# 2. Phân tích cặp số và bộ ba số
pair_counts = Counter()
triple_counts = Counter()
for result in df['Kết Quả']:
    pairs = list(combinations(sorted(result), 2))
    pair_counts.update(pairs)
    triples = list(combinations(sorted(result), 3))
    triple_counts.update(triples)

# Chuẩn hóa tần suất
num_draws = len(df)
top_pairs = pd.DataFrame(pair_counts.most_common(5), columns=['Cặp số', 'Tần suất'])
top_pairs['Tần suất (%)'] = top_pairs['Tần suất'] / num_draws * 100
print("\nTop 5 cặp số xuất hiện cùng nhau nhiều nhất:")
print(top_pairs[['Cặp số', 'Tần suất', 'Tần suất (%)']])

top_triples = pd.DataFrame(triple_counts.most_common(5), columns=['Bộ ba số', 'Tần suất'])
top_triples['Tần suất (%)'] = top_triples['Tần suất'] / num_draws * 100
print("\nTop 5 bộ ba số xuất hiện cùng nhau nhiều nhất:")
print(top_triples[['Bộ ba số', 'Tần suất', 'Tần suất (%)']])

# 3. Phân tích xu hướng Jackpot
plt.figure(figsize=(14, 7))
plt.plot(df['Ngày'], df['Jackpot'] / 1e9, label='Jackpot (tỷ VND)', color='blue')
plt.title('Xu hướng giá trị Jackpot theo thời gian', fontsize=14)
plt.xlabel('Ngày', fontsize=12)
plt.ylabel('Jackpot (tỷ VND)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'jackpot_trend.png'), dpi=300)
plt.close()

# 4. Phân tích tăng/giảm Jackpot
df['Jackpot_Change'] = df['Jackpot'].diff()
df['Trend'] = df['Jackpot_Change'].apply(lambda x: 'Tăng' if x > 0 else 'Giảm' if x < 0 else 'Không đổi')
trend_counts = df['Trend'].value_counts()
print("\nTần suất tăng/giảm Jackpot:")
print(trend_counts)

avg_increase = df[df['Jackpot_Change'] > 0]['Jackpot_Change'].mean() / 1e9
avg_decrease = df[df['Jackpot_Change'] < 0]['Jackpot_Change'].mean() / 1e9
print(f"\nGiá trị tăng trung bình: {avg_increase:.2f} tỷ VND")
print(f"Giá trị giảm trung bình: {avg_decrease:.2f} tỷ VND")

# 5. Phân tích chu kỳ Jackpot
df['Jackpot_Reset'] = df['Jackpot_Change'] < -10e9  # Giả định giảm lớn là reset Jackpot
cycles = []
current_cycle = 0
for reset in df['Jackpot_Reset']:
    if reset:
        cycles.append(current_cycle)
        current_cycle = 0
    else:
        current_cycle += 1
if current_cycle > 0:
    cycles.append(current_cycle)

print("\nPhân tích chu kỳ Jackpot:")
print(f"- Số chu kỳ: {len(cycles)}")
print(f"- Độ dài chu kỳ trung bình: {np.mean(cycles):.2f} kỳ")
print(f"- Độ dài chu kỳ tối đa: {max(cycles)} kỳ")
print(f"- Độ dài chu kỳ tối thiểu: {min(cycles)} kỳ")

C:\Users\NITRO 5\AppData\Local\Temp\ipykernel_18208\3745541686.py:23: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=freq.index, y=freq.values, palette='viridis')



Top 5 cặp số xuất hiện cùng nhau nhiều nhất:
     Cặp số  Tần suất  Tần suất (%)
0  (10, 22)        34      2.507375
1  (13, 20)        34      2.507375
2   (7, 44)        33      2.433628
3   (4, 37)        33      2.433628
4  (19, 24)        32      2.359882

Top 5 bộ ba số xuất hiện cùng nhau nhiều nhất:
       Bộ ba số  Tần suất  Tần suất (%)
0  (11, 26, 28)         9      0.663717
1  (10, 22, 36)         9      0.663717
2    (1, 7, 16)         9      0.663717
3  (25, 35, 39)         8      0.589971
4  (10, 22, 43)         8      0.589971

Tần suất tăng/giảm Jackpot:
Trend
Không đổi    1356
Name: count, dtype: int64

Giá trị tăng trung bình: nan tỷ VND
Giá trị giảm trung bình: nan tỷ VND

Phân tích chu kỳ Jackpot:
- Số chu kỳ: 1
- Độ dài chu kỳ trung bình: 1356.00 kỳ
- Độ dài chu kỳ tối đa: 1356 kỳ
- Độ dài chu kỳ tối thiểu: 1356 kỳ


In [20]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import json
import os

# Tận dụng df từ cell trước
# Kiểm tra dữ liệu
if 'Kết Quả' not in df.columns:
    raise ValueError("Dữ liệu thiếu cột 'Kết Quả'")

# Chuyển đổi Kết Quả
df['Kết Quả'] = df['Kết Quả'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)

# Kiểm tra tính hợp lệ
df['Valid_Result'] = df['Kết Quả'].apply(validate_result)
invalid_results = df[~df['Valid_Result'].apply(lambda x: x[0])]
if not invalid_results.empty:
    raise ValueError("Dữ liệu chứa kết quả không hợp lệ, vui lòng kiểm tra.")

# Mã hóa dữ liệu
mlb = MultiLabelBinarizer(classes=range(1, 46))
y_encoded = mlb.fit_transform(df['Kết Quả'])

# Tạo chuỗi time-series
sequence_length = 50
X, y = [], []
for i in range(len(y_encoded) - sequence_length):
    X.append(y_encoded[i:i + sequence_length])
    y.append(y_encoded[i + sequence_length])

X = np.array(X)  # (samples, sequence_length, 45)
y = np.array(y)  # (samples, 45)
print(f"Tổng số mẫu học: {len(X)}")

# Lưu dữ liệu mã hóa
np.save(os.path.join(OUTPUT_DIR, 'X.npy'), X)
np.save(os.path.join(OUTPUT_DIR, 'y.npy'), y)
print(f"Đã lưu dữ liệu mã hóa: X.npy, y.npy")

# Chia tập huấn luyện và kiểm tra
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
print(f"Kích thước tập huấn luyện: {X_train.shape}")
print(f"Kích thước tập kiểm tra: {X_test.shape}")

Tổng số mẫu học: 1306
Đã lưu dữ liệu mã hóa: X.npy, y.npy
Kích thước tập huấn luyện: (1044, 50, 45)
Kích thước tập kiểm tra: (262, 50, 45)


In [21]:
import pandas as pd

# Đọc dữ liệu
df = pd.read_csv(r'E:\vietlot_mega\data_645/vietlott_645_clean.csv')

# Đếm số kỳ quay
num_draws = len(df)

print(f"Số kỳ quay trong dữ liệu: {num_draws}")

Số kỳ quay trong dữ liệu: 1356


In [ ]:
def load_and_preprocess_data(file_path='E:/vietlot_mega\data_645/vietlott_645_clean.csv'):
    logger.info(f"Đang tải dữ liệu từ {file_path}...")
    
    # Kiểm tra file tồn tại
    if not os.path.exists(file_path):
        logger.error(f"File {file_path} không tồn tại.")
        raise FileNotFoundError(f"File {file_path} không tồn tại.")
    
    # Đọc dữ liệu
    df = pd.read_csv(file_path, encoding='utf-8')
    if not all(col in df.columns for col in ['Ngày', 'Kết Quả']):
        logger.error("File CSV thiếu cột 'Ngày' hoặc 'Kết Quả'.")
        raise KeyError("File CSV thiếu cột cần thiết.")
    
    # Chuyển đổi cột Kết Quả
    def parse_results(res):
        try:
            if isinstance(res, str):
                numbers = json.loads(res) if '[' in res else [int(x) for x in res.split()]
            else:
                numbers = res
            if len(numbers) == 6 and all(isinstance(x, int) and 1 <= x <= 45 for x in numbers):
                return sorted(numbers)
            return None
        except Exception:
            return None
    
    initial_rows = len(df)
    df['Numbers'] = df['Kết Quả'].apply(parse_results)
    df = df.dropna(subset=['Numbers'])
    logger.info(f"Loại bỏ {initial_rows - len(df)} hàng do kết quả không hợp lệ.")
    
    # Chuyển đổi ngày
    df['Date'] = pd.to_datetime(df['Ngày'], format='%Y-%m-%d', errors='coerce')
    df = df.dropna(subset=['Date']).sort_values('Date')
    
    # Thêm đặc trưng
    df['Draw'] = range(1, len(df) + 1)
    df['DayOfWeek'] = df['Date'].dt.dayofweek
    df['Sum'] = df['Numbers'].apply(sum)
    df['OddCount'] = df['Numbers'].apply(lambda x: sum(1 for n in x if n % 2 == 1))
    df['Range'] = df['Numbers'].apply(lambda x: max(x) - min(x))
    
    logger.info(f"Đã tải {len(df)} kết quả hợp lệ.")
    return df

# Tải dữ liệu
df = load_and_preprocess_data()

2025-05-30 16:27:09,543 - INFO - Đang tải dữ liệu từ E:/vietlot_mega\data_645/vietlott_645_clean.csv...
2025-05-30 16:27:09,563 - INFO - Loại bỏ 0 hàng do kết quả không hợp lệ.
2025-05-30 16:27:09,569 - INFO - Đã tải 1356 kết quả hợp lệ.


# LSTM + GRU + Random Fores + Transformer + Tần xuất

In [ ]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization, MultiHeadAttention, Input, Flatten, LSTM, GRU, Add
from tensorflow.keras.callbacks import EarlyStopping, Callback, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.metrics import Precision
from tensorflow.keras.regularizers import l2
import tensorflow as tf
import random
from collections import Counter
from datetime import datetime, timedelta
import os
import logging
import warnings
import itertools

warnings.filterwarnings('ignore')

# Thiết lập logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Thiết lập seed
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# Hằng số cấu hình
FILE_PATH = 'E:\\vietlot_mega\\data_655\\vietlott_655_clean.csv'
SEQUENCE_LENGTH = 30
CONFIG = {
    'num_heads': 4,
    'ff_dim': 128,
    'learning_rate': 0.001,
    'batch_size': 32,
    'epochs': 150,
    'dropout_rate': 0.3,
    'hist_ratio': 0.3,
    'lstm_units': 64,
    'gru_units': 64,
    'rf_n_estimators': 100
}

# Callback
class PrintTrainingMetrics(Callback):
    def on_epoch_end(self, epoch, logs=None):
        logging.info(f"Epoch {epoch+1}/{self.params['epochs']} - "
                     f"loss: {logs['loss']:.4f} - val_loss: {logs['val_loss']:.4f} - "
                     f"precision_at_k: {logs.get('precision_at_k', 0):.4f} - "
                     f"val_precision_at_k: {logs.get('val_precision_at_k', 0):.4f}")

class VietlottPredictor:
    def __init__(self, sequence_length=SEQUENCE_LENGTH, file_path=FILE_PATH, config=CONFIG):
        self.sequence_length = sequence_length
        self.file_path = file_path
        self.config = config
        self.results = None
        self.df = None
        self.scaler = None
        self.feature_scaler = None
        self.X_train, self.X_test, self.y_train, self.y_test = None, None, None, None
        self.last_sequence = None
        self.statistical_patterns = {}
        self.next_draw = None
        self.next_date = None
        self.rf_model = None

    def get_next_draw(self, steps_ahead=1):
        if self.df is None or self.df.empty:
            raise ValueError("Dữ liệu chưa được tải.")
        logging.info("Xác định kỳ quay và ngày quay tiếp theo...")
        self.df['Draw'] = range(1, len(self.df) + 1)
        last_draw = self.df['Draw'].iloc[-1]
        last_date = self.df['Date'].iloc[-1]
        logging.info(f"Ngày cuối trong dữ liệu: {last_date.strftime('%Y-%m-%d')} (Thứ {last_date.weekday()+1})")
        target_days = [1, 3, 5]  # Thứ Ba, thứ Năm, thứ Bảy
        current_date = last_date
        for _ in range(steps_ahead):
            current_day = current_date.weekday()
            days_ahead = min((target - current_day) % 7 or 7 for target in target_days)
            current_date += timedelta(days=days_ahead)
        self.next_date = current_date
        self.next_draw = last_draw + steps_ahead
        logging.info(f"Kỳ quay tiếp theo: {self.next_draw}, Ngày: {self.next_date.strftime('%Y-%m-%d')}")
        return self.next_draw, self.next_date

    def compute_statistical_patterns(self):
        logging.info("Tính toán các mẫu thống kê...")
        all_numbers = np.array([num for draw in self.results for num in draw])
        total_draws = len(self.results)

        freq = np.bincount(all_numbers, minlength=56)[1:]
        self.statistical_patterns['frequencies'] = {i: int(f) for i, f in enumerate(freq, 1)}
        self.statistical_patterns['number_probabilities'] = {i: f / (total_draws * 6) for i, f in enumerate(freq, 1)}
        candidate_numbers = (np.argsort(freq)[::-1][:20] + 1).tolist()
        self.statistical_patterns['candidate_numbers'] = candidate_numbers if candidate_numbers else list(range(1, 56))

        recent_numbers = np.array([num for draw in self.results[-15:] for num in draw])
        recent_freq = np.bincount(recent_numbers, minlength=56)[1:]
        self.statistical_patterns['recent_frequencies'] = {i: f / 90 for i, f in enumerate(recent_freq, 1)}
        recent_top_10 = (np.argsort(recent_freq)[::-1][:10] + 1).tolist()
        self.statistical_patterns['recent_top_10'] = recent_top_10

        pair_counts = Counter()
        for draw in self.results:
            for pair in itertools.combinations(sorted(draw), 2):
                pair_counts[pair] += 1
        self.statistical_patterns['pair_probabilities'] = {k: v / total_draws for k, v in pair_counts.items()}

        last_appearance = {}
        for i, draw in enumerate(reversed(self.results)):
            for num in draw:
                if num not in last_appearance:
                    last_appearance[num] = i
        self.statistical_patterns['last_appearance'] = last_appearance

        recent_momentum = {}
        recent_draws = self.results[-5:]
        for num in range(1, 56):
            momentum = sum(1 for draw in recent_draws if num in draw) / 5
            recent_momentum[num] = momentum
        self.statistical_patterns['recent_momentum'] = recent_momentum

        logging.debug(f"Candidate numbers: {self.statistical_patterns['candidate_numbers']}")
        logging.debug(f"Recent top 10: {self.statistical_patterns['recent_top_10']}")

    def load_and_preprocess_data(self):
        logging.info("Đang tải và tiền xử lý dữ liệu...")
        try:
            if not os.path.exists(self.file_path):
                logging.error(f"File không tồn tại: {self.file_path}")
                raise FileNotFoundError(f"File không tồn tại: {self.file_path}")
            try:
                self.df = pd.read_csv(self.file_path, encoding='utf-8')
            except UnicodeDecodeError:
                logging.warning("Không thể đọc file với UTF-8, thử encoding latin1...")
                self.df = pd.read_csv(self.file_path, encoding='latin1')
            expected_columns = ['Ngày', 'Kết Quả']
            if not all(col in self.df.columns for col in expected_columns):
                logging.error(f"Thiếu cột cần thiết. Cột có sẵn: {self.df.columns.tolist()}")
                raise ValueError(f"CSV phải chứa các cột: {expected_columns}")
            self.df['Numbers'] = self.df['Kết Quả'].apply(literal_eval)
            self.df = self.df.dropna(subset=['Numbers'])
            self.df['Date'] = pd.to_datetime(self.df['Ngày'], format='%Y-%m-%d', errors='coerce')
            self.df = self.df.dropna(subset=['Date']).sort_values('Date')
            logging.info("5 dòng đầu của dữ liệu:")
            logging.info(f"\n{self.df.head().to_string()}")
            logging.info("5 dòng cuối của dữ liệu:")
            logging.info(f"\n{self.df.tail().to_string()}")
            if len(self.df) < 600:
                logging.error(f"Dữ liệu chỉ có {len(self.df)} kỳ, cần ít nhất 600 kỳ.")
                raise ValueError(f"Dữ liệu chỉ có {len(self.df)} kỳ, cần ít nhất 600 kỳ.")
            if len(set([n for draw in self.df['Numbers'] for n in draw])) < 55:
                logging.error("Dữ liệu không chứa đủ 55 số khác nhau.")
                raise ValueError("Dữ liệu không chứa đủ 55 số khác nhau.")
            if (self.df['Date'].max() - self.df['Date'].min()).days < 1095:
                logging.error("Dữ liệu cần trải dài ít nhất 3 năm.")
                raise ValueError("Dữ liệu cần trải dài ít nhất 3 năm.")
            self.results = self.df['Numbers'].tolist()
            self.compute_statistical_patterns()
            self.df['OddCount'] = self.df['Numbers'].apply(lambda x: sum(1 for n in x if n % 2 == 1))
            self.df['Sum'] = self.df['Numbers'].apply(lambda x: np.log1p(sum(x)))
            self.df['Range'] = self.df['Numbers'].apply(lambda x: np.log1p(max(x) - min(x)))
            self.df['ClusterCount'] = self.df['Numbers'].apply(lambda x: sum(1 for n in x if 1 <= n <= 18) / 6 + sum(1 for n in x if 19 <= n <= 36) / 6 + sum(1 for n in x if 37 <= n <= 55) / 6)
            self.df['RecentFreq'] = self.df['Numbers'].apply(lambda x: sum(self.statistical_patterns['recent_frequencies'].get(n, 0) for n in x))
            self.df['ConsecutiveCount'] = self.df['Numbers'].apply(lambda x: sum(1 for i in range(len(x)-1) if sorted(x)[i+1] == sorted(x)[i] + 1))
            self.df['DeltaSum'] = self.df['Sum'].diff().fillna(0)
            self.df['NumberGaps'] = self.df['Numbers'].apply(lambda x: np.mean([sorted(x)[i+1] - sorted(x)[i] for i in range(len(x)-1)]))
            self.df['RecentHotSpot'] = self.df['Numbers'].apply(lambda x: sum(1 for n in x if n in [num for draw in self.results[-5:] for num in draw]) / 6)
            self.df['PatternScore'] = self.df['Numbers'].apply(lambda x: sum(self.statistical_patterns['pair_probabilities'].get(tuple(sorted([a, b])), 0) for a, b in itertools.combinations(x, 2)))
            self.df['LastAppearance'] = self.df['Numbers'].apply(lambda x: np.mean([self.statistical_patterns['last_appearance'].get(n, len(self.results)) for n in x]))
            self.df['RecentMomentum'] = self.df['Numbers'].apply(lambda x: np.mean([self.statistical_patterns['recent_momentum'].get(n, 0) for n in x]))
            logging.info(f"Dữ liệu đã được tải và tiền xử lý thành công. Số kỳ: {len(self.df)}")
        except Exception as e:
            logging.error(f"Lỗi khi tải dữ liệu: {str(e)}")
            raise

    def prepare_sequences(self):
        logging.info("Chuẩn bị chuỗi dữ liệu...")
        try:
            sequences = []
            targets = []
            features = []
            for i in range(len(self.results) - self.sequence_length):
                seq = self.results[i:i + self.sequence_length]
                target = self.results[i + self.sequence_length]
                seq_features = [
                    self.df['OddCount'].iloc[i:i + self.sequence_length].values,
                    self.df['Sum'].iloc[i:i + self.sequence_length].values,
                    self.df['Range'].iloc[i:i + self.sequence_length].values,
                    self.df['ClusterCount'].iloc[i:i + self.sequence_length].values,
                    self.df['RecentFreq'].iloc[i:i + self.sequence_length].values,
                    self.df['ConsecutiveCount'].iloc[i:i + self.sequence_length].values,
                    self.df['DeltaSum'].iloc[i:i + self.sequence_length].values,
                    self.df['NumberGaps'].iloc[i:i + self.sequence_length].values,
                    self.df['RecentHotSpot'].iloc[i:i + self.sequence_length].values,
                    self.df['PatternScore'].iloc[i:i + self.sequence_length].values,
                    self.df['LastAppearance'].iloc[i:i + self.sequence_length].values,
                    self.df['RecentMomentum'].iloc[i:i + self.sequence_length].values
                ]
                sequences.append(seq)
                targets.append(target)
                features.append(np.stack(seq_features, axis=-1))
            X = np.array(sequences)
            X_features = np.array(features)
            y = np.array(targets)
            self.scaler = MinMaxScaler(feature_range=(0, 1))
            X_reshaped = X.reshape(-1, X.shape[-1])
            X_scaled = self.scaler.fit_transform(X_reshaped)
            X = X_scaled.reshape(X.shape)
            self.feature_scaler = MinMaxScaler(feature_range=(0, 1))
            X_features_reshaped = X_features.reshape(-1, X_features.shape[-1])
            X_features_scaled = self.feature_scaler.fit_transform(X_features_reshaped)
            X_features = X_features_scaled.reshape(X_features.shape)
            X = np.concatenate([X, X_features], axis=-1)
            from sklearn.model_selection import train_test_split
            self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
                X, y, test_size=0.15, random_state=RANDOM_SEED, shuffle=False
            )
            logging.info(f"Kích thước tập huấn luyện: {len(self.X_train)}, tập kiểm tra: {len(self.X_test)}")
            last_sequence = np.array(self.results[-self.sequence_length:])
            last_features = np.stack([
                self.df['OddCount'].iloc[-self.sequence_length:].values,
                self.df['Sum'].iloc[-self.sequence_length:].values,
                self.df['Range'].iloc[-self.sequence_length:].values,
                self.df['ClusterCount'].iloc[-self.sequence_length:].values,
                self.df['RecentFreq'].iloc[-self.sequence_length:].values,
                self.df['ConsecutiveCount'].iloc[-self.sequence_length:].values,
                self.df['DeltaSum'].iloc[-self.sequence_length:].values,
                self.df['NumberGaps'].iloc[-self.sequence_length:].values,
                self.df['RecentHotSpot'].iloc[-self.sequence_length:].values,
                self.df['PatternScore'].iloc[-self.sequence_length:].values,
                self.df['LastAppearance'].iloc[-self.sequence_length:].values,
                self.df['RecentMomentum'].iloc[-self.sequence_length:].values
            ], axis=-1)
            last_sequence_reshaped = last_sequence.reshape(-1, last_sequence.shape[-1])
            last_sequence_scaled = self.scaler.transform(last_sequence_reshaped).reshape(1, self.sequence_length, 6)
            last_features_reshaped = last_features.reshape(-1, last_features.shape[-1])
            last_features_scaled = self.feature_scaler.transform(last_features_reshaped).reshape(1, self.sequence_length, 12)
            self.last_sequence = np.concatenate([last_sequence_scaled, last_features_scaled], axis=-1)
        except Exception as e:
            logging.error(f"Lỗi khi chuẩn bị chuỗi: {str(e)}")
            raise

    def positional_encoding(self, seq_len, d_model):
        position = np.arange(seq_len)[:, np.newaxis]
        div_term = np.exp(np.arange(0, d_model, 2) * (-np.log(10000.0) / d_model))
        pos_encoding = np.zeros((seq_len, d_model))
        pos_encoding[:, 0::2] = np.sin(position * div_term)
        pos_encoding[:, 1::2] = np.cos(position * div_term)
        return tf.cast(pos_encoding[np.newaxis, ...], dtype=tf.float32)

    def build_lstm_model(self):
        inputs = Input(shape=(self.sequence_length, 18))
        x = LSTM(self.config['lstm_units'], return_sequences=True)(inputs)
        x = Dropout(self.config['dropout_rate'])(x)
        x = LSTM(self.config['lstm_units'])(x)
        x = Dropout(self.config['dropout_rate'])(x)
        x = Dense(128, activation='relu')(x)
        outputs = Dense(55, activation='sigmoid')(x)
        model = Model(inputs, outputs)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=self.config['learning_rate']),
                      loss='binary_crossentropy', metrics=[Precision(top_k=6, name='precision_at_k')])
        return model

    def build_gru_model(self):
        inputs = Input(shape=(self.sequence_length, 18))
        x = GRU(self.config['gru_units'], return_sequences=True)(inputs)
        x = Dropout(self.config['dropout_rate'])(x)
        x = GRU(self.config['gru_units'])(x)
        x = Dropout(self.config['dropout_rate'])(x)
        x = Dense(128, activation='relu')(x)
        outputs = Dense(55, activation='sigmoid')(x)
        model = Model(inputs, outputs)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=self.config['learning_rate']),
                      loss='binary_crossentropy', metrics=[Precision(top_k=6, name='precision_at_k')])
        return model

    def build_transformer_model(self):
        inputs = Input(shape=(self.sequence_length, 18))
        pos_encoding = self.positional_encoding(self.sequence_length, 18)
        x = Add()([inputs, pos_encoding])
        x = MultiHeadAttention(num_heads=self.config['num_heads'], key_dim=64)(x, x)
        x = Dropout(self.config['dropout_rate'])(x)
        x = LayerNormalization(epsilon=1e-6)(x + inputs)
        ff = Dense(self.config['ff_dim'], activation='relu')(x)
        ff = Dense(inputs.shape[-1])(ff)
        x = Dropout(self.config['dropout_rate'])(ff)
        x = LayerNormalization(epsilon=1e-6)(x + ff)
        x = Flatten()(x)
        x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)
        x = Dropout(self.config['dropout_rate'])(x)
        outputs = Dense(55, activation='sigmoid')(x)
        model = Model(inputs, outputs)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=self.config['learning_rate']),
                      loss='binary_crossentropy', metrics=[Precision(top_k=6, name='precision_at_k')])
        return model

    def train_rf_model(self):
        logging.info("Huấn luyện Random Forest...")
        y_train_rf = np.zeros((len(self.y_train), 55))
        for i, draw in enumerate(self.y_train):
            for num in draw:
                y_train_rf[i, num-1] = 1

        X_train_features = self.X_train[:, :, 6:]
        X_train_rf = X_train_features.reshape(X_train_features.shape[0], -1)

        self.rf_model = RandomForestClassifier(n_estimators=self.config['rf_n_estimators'], random_state=RANDOM_SEED)
        self.rf_model.fit(X_train_rf, y_train_rf)

    def optimize_numbers(self, nums, probs, iteration=0):
        """Tối ưu hóa bộ số với sự đa dạng hơn."""
        logging.debug(f"Optimizing input (iteration {iteration}): {nums}")
        nums = sorted(list(set(nums)))
        target_len = 6

        # Bổ sung số ngẫu nhiên để tạo sự đa dạng
        available = [n for n in self.statistical_patterns.get('recent_top_10', []) if n not in nums]
        if len(available) < target_len - len(nums):
            available.extend([n for n in self.statistical_patterns.get('candidate_numbers', []) if n not in nums and n not in available])
        while len(nums) < target_len:
            if not available:
                available = [n for n in range(1, 56) if n not in nums]
            # Chọn ngẫu nhiên với xác suất dựa trên tần suất
            probs_available = [self.statistical_patterns.get('recent_frequencies', {}).get(n, 0) for n in available]
            probs_available = [p if p > 0 else 0.01 for p in probs_available]  # Đảm bảo không có xác suất 0
            probs_available = np.array(probs_available) / sum(probs_available)
            chosen = np.random.choice(available, p=probs_available)
            nums.append(chosen)
            nums = sorted(list(set(nums)))
            available = [n for n in available if n != chosen]

        # Tối ưu hóa với nhiều lần lặp
        best_nums = nums.copy()
        best_score = sum(probs[n-1] + self.statistical_patterns.get('recent_frequencies', {}).get(n, 0) +
                         3.0 * self.statistical_patterns.get('recent_momentum', {}).get(n, 0) -
                         0.1 * self.statistical_patterns.get('last_appearance', {}).get(n, len(self.results))
                         for n in nums)

        for _ in range(30):  # Tăng số lần lặp để thử nhiều tổ hợp hơn
            temp_nums = nums.copy()
            idx = random.randint(0, len(temp_nums) - 1)
            available = [n for n in self.statistical_patterns.get('recent_top_10', []) if n not in temp_nums] or \
                        [n for n in range(1, 56) if n not in temp_nums]
            if available:
                # Chọn số mới với xác suất dựa trên tần suất
                probs_available = [self.statistical_patterns.get('recent_frequencies', {}).get(n, 0) for n in available]
                probs_available = [p if p > 0 else 0.01 for p in probs_available]
                probs_available = np.array(probs_available) / sum(probs_available)
                new_num = np.random.choice(available, p=probs_available)
                temp_nums[idx] = new_num
                temp_nums = sorted(list(set(temp_nums)))

            total = sum(temp_nums)
            odd_count = sum(1 for n in temp_nums if n % 2 == 1)
            range_dist = (any(1 <= n <= 18 for n in temp_nums) and
                          any(19 <= n <= 36 for n in temp_nums) and
                          any(37 <= n <= 55 for n in temp_nums))
            consecutive_count = sum(1 for i in range(len(temp_nums)-1) if temp_nums[i+1] == temp_nums[i] + 1)
            recent_top_count = sum(1 for n in temp_nums if n in self.statistical_patterns.get('recent_top_10', []))
            last_appearance_count = sum(1 for n in temp_nums if self.statistical_patterns.get('last_appearance', {}).get(n, len(self.results)) <= 5)

            if (150 <= total <= 190 and odd_count in [3, 4] and range_dist and
                consecutive_count <= 2 and recent_top_count >= 4 and last_appearance_count >= 3):
                score = sum(probs[n-1] + self.statistical_patterns.get('recent_frequencies', {}).get(n, 0) +
                            3.0 * self.statistical_patterns.get('recent_momentum', {}).get(n, 0) -
                            0.1 * self.statistical_patterns.get('last_appearance', {}).get(n, len(self.results))
                            for n in temp_nums)
                # Thêm yếu tố ngẫu nhiên để tránh bị kẹt
                if score > best_score or (score > best_score * 0.95 and random.random() < 0.3):
                    best_score = score
                    best_nums = temp_nums.copy()

        logging.debug(f"Optimized output (iteration {iteration}): {best_nums}")
        return best_nums[:6]

    def train_models(self):
        callbacks = [
            EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6),
            ModelCheckpoint('best_model.h5', monitor='val_precision_at_k', save_best_only=True, mode='max'),
            PrintTrainingMetrics()
        ]

        logging.info("Huấn luyện mô hình LSTM...")
        lstm_model = self.build_lstm_model()
        y_train_bin = np.array([[1 if j+1 in draw else 0 for j in range(55)] for draw in self.y_train], dtype=np.float32)
        y_test_bin = np.array([[1 if j+1 in draw else 0 for j in range(55)] for draw in self.y_test], dtype=np.float32)
        lstm_model.fit(self.X_train, y_train_bin, validation_data=(self.X_test, y_test_bin),
                       epochs=self.config['epochs'], batch_size=self.config['batch_size'], callbacks=callbacks, verbose=0)

        logging.info("Huấn luyện mô hình GRU...")
        gru_model = self.build_gru_model()
        gru_model.fit(self.X_train, y_train_bin, validation_data=(self.X_test, y_test_bin),
                      epochs=self.config['epochs'], batch_size=self.config['batch_size'], callbacks=callbacks, verbose=0)

        logging.info("Huấn luyện mô hình Transformer...")
        transformer_model = self.build_transformer_model()
        transformer_model.fit(self.X_train, y_train_bin, validation_data=(self.X_test, y_test_bin),
                              epochs=self.config['epochs'], batch_size=self.config['batch_size'], callbacks=callbacks, verbose=0)

        self.train_rf_model()

        return lstm_model, gru_model, transformer_model

    def predict_ensemble(self, n_predictions=1):
        logging.info(f"Dự đoán {n_predictions} bộ số...")
        try:
            lstm_model, gru_model, transformer_model = self.train_models()

            lstm_probs = lstm_model.predict(self.last_sequence, verbose=0)[0]
            gru_probs = gru_model.predict(self.last_sequence, verbose=0)[0]
            transformer_probs = transformer_model.predict(self.last_sequence, verbose=0)[0]

            last_features = self.last_sequence[0, :, 6:].reshape(1, -1)
            rf_probs = self.rf_model.predict_proba(last_features)[0]

            final_probs = (0.3 * lstm_probs + 0.3 * gru_probs + 0.3 * transformer_probs + 0.1 * rf_probs[:, 1]) / 4
            final_probs = final_probs / final_probs.sum() if final_probs.sum() > 0 else np.ones(55) / 55

            stat_probs = np.array([self.statistical_patterns.get('recent_frequencies', {}).get(i+1, 0) for i in range(55)])
            final_probs = (0.7 * final_probs + 0.3 * stat_probs) / (0.7 + 0.3)

            candidates = []
            for i in range(n_predictions):
                # Chọn ngẫu nhiên 10 số ban đầu từ top 20 để tăng tính đa dạng
                top_20 = np.argsort(final_probs)[-20:][::-1] + 1
                initial_nums = random.sample(top_20.tolist(), 10)
                nums = self.optimize_numbers(initial_nums, final_probs, iteration=i)
                candidates.append(nums)

            df_predictions = pd.DataFrame(candidates, columns=[f'Số {i+1}' for i in range(6)])
            df_predictions.to_csv('predictions.csv', index=False)
            logging.info("Đã lưu dự đoán vào predictions.csv")
            return candidates
        except Exception as e:
            logging.error(f"Lỗi khi dự đoán: {str(e)}")
            raise

if __name__ == "__main__":
    try:
        predictor = VietlottPredictor()
        predictor.load_and_preprocess_data()
        next_draw, next_date = predictor.get_next_draw(steps_ahead=2)
        predictor.prepare_sequences()
        predictions = predictor.predict_ensemble(n_predictions=2)
        print(f"\nDự đoán cho kỳ quay {next_draw} (ngày {next_date.strftime('%Y-%m-%d')} - Hôm nay, 31/05/2025):")
        for i, pred in enumerate(predictions, 1):
            print(f"Bộ {i}: {pred}")
    except Exception as e:
        logging.error(f"Lỗi chính: {str(e)}")

2025-05-31 15:02:39,091 - INFO - Đang tải và tiền xử lý dữ liệu...
2025-05-31 15:02:39,117 - INFO - 5 dòng đầu của dữ liệu:
2025-05-31 15:02:39,122 - INFO - 
            Ngày                   Kết Quả                   Numbers       Date
1194  2017-08-01   [5, 10, 14, 23, 24, 38]   [5, 10, 14, 23, 24, 38] 2017-08-01
1193  2017-08-03    [4, 9, 24, 25, 27, 45]    [4, 9, 24, 25, 27, 45] 2017-08-03
1192  2017-08-05    [1, 5, 11, 32, 40, 45]    [1, 5, 11, 32, 40, 45] 2017-08-05
1191  2017-08-08  [19, 36, 39, 41, 46, 51]  [19, 36, 39, 41, 46, 51] 2017-08-08
1190  2017-08-10  [10, 11, 19, 41, 50, 54]  [10, 11, 19, 41, 50, 54] 2017-08-10
2025-05-31 15:02:39,123 - INFO - 5 dòng cuối của dữ liệu:
2025-05-31 15:02:39,128 - INFO - 
         Ngày                   Kết Quả                   Numbers       Date
4  2025-05-17    [2, 7, 26, 29, 41, 50]    [2, 7, 26, 29, 41, 50] 2025-05-17
3  2025-05-20  [19, 27, 44, 45, 47, 52]  [19, 27, 44, 45, 47, 52] 2025-05-20
2  2025-05-22    [3, 9, 14, 41, 47, 55]


Dự đoán cho kỳ quay 1197 (ngày 2025-05-31 - Hôm nay, 31/05/2025):
Bộ 1: [3, 15, 19, 39, 41, 44]
Bộ 2: [3, 15, 19, 39, 41, 44]
